In [ ]:
#=====================================
# Libraries
#=====================================
import xarray as xr
import os
import pandas as pd
import statistics
#=====================================================
# Preprocessing the data
#=====================================================

# Directory of the project
basedir = "C:/Users/ilung/Documents/Jedha_bootcamp/capstone_project/data"

# Importing the spreadsheet containing the name and geographic coordinates of cities
cities = pd.read_excel(f"{basedir}/raw/communes_paca_wgs84.xlsx")
cities_list = cities["Name"].to_list()
cities_long = cities["longitude"].to_list()
cities_lat = cities["latitude"].to_list()

# Variables to be used in the preprocessing
pollutants_list = ["no2", "pm2p5", "pm10", "o3", "so2"] # Also the names used in the folders, files names and netcdf variables
months_list = [str(x).zfill(2) for x in range(1, 13)] # Months from 01 to 12. Used in files name
years_list = [str(x) for x in list(range(2013, 2023))] # From 2013 to 2022

# Preprocessing
for year in years_list: 
    all_pollutants = [] # The yearly average concentration of the pollutants
    
    for pollutant in pollutants_list:
        year_average_cities = [] # Yearly average for all the cities for a single polluant

        for city_name, longitude, latitude in zip(cities_list, cities_long, cities_lat):
            hourly_value_year = [] # Monthly hourly data

            for month in months_list:
                file = f"{basedir}/raw/{year}/{pollutant}/cams.eaq.vra.ENSa.{pollutant}.l0.{year}-{month}.nc"
                dataset = xr.open_dataset(file, engine="netcdf4")
                hourly_value_month = list(dataset[pollutant].sel(lon = longitude , lat = latitude, method = "nearest").values) # Single city. Single pollutant
                hourly_value_year.extend(hourly_value_month) # Single city. Single pollutant.

            year_average = statistics.mean(hourly_value_year) # Single city. Single pollutant.
            year_average_cities.append(year_average) # List of concentrations. All cities. Single pollutant 

        all_pollutants.append(year_average_cities) # List of lists
    pollutant_df = pd.DataFrame(list(map(list, zip(*all_pollutants))), columns=pollutants_list)
    final_df = pd.concat([cities, pollutant_df], axis=1)
    final_df.to_excel(f"{basedir}/clean/{year}.xlsx", index=False)
